In [ ]:
import sys
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append('../wakis/')

from wakis import SolverFIT3D
from wakis import GridFIT3D 
from wakis import WakeSolver

# ---------- Domain setup ---------
# Number of mesh cells
Nx = 100
Ny = 100
Nz = 161
#dt = 2.187760221e-12 # CST

# Embedded boundaries
stl_cavity = 'stl/cavity_vacuum.stl' 
stl_shell = 'stl/cavity_shell.stl'
stl_letters = 'stl/CEI_stl_readex_pro_700.stl'

stl_solids = {'cavity': stl_cavity, 
              'shell': stl_shell,
              'letters' : stl_letters}

stl_materials = {'cavity': 'vacuum', 
                 'shell': [30, 1.0, 30],
                 'letters' : 'pec'}

stl_rotate = {'cavity': [0., 0., 0.],
                'shell': [0., 0., 0.],
                'letters' : [90, -90, 0]}

stl_translate = {'cavity': [0., 0., 0.],
                 'shell': [0., 0., 0.],
                 'letters' : [50, 0, 0]}


stl_scale = {'cavity': [1., 1., 1.],
                 'shell': [1., 1., 1.],
                 'letters' : [1.e-3, 1.e-3, 1.e-3]} #[0.7e-2, 0.4e-2, 0.4e-2]}

# Domain bounds
letters = pv.read(stl_letters) \
        .rotate_x(stl_rotate['letters'][0]) \
        .rotate_y(stl_rotate['letters'][1]) \
        .rotate_z(stl_rotate['letters'][2]) \
        .translate(stl_translate['letters']) \
        .scale(stl_scale['letters'])

surf = pv.read(stl_shell)+letters

Inspect the imported geometry using interactive plotting:

In [ ]:
p = pv.Plotter()
p.add_mesh_clip_box(surf, color='white', rotation_enabled=False)
p.add_axes()
p.camera_position = 'zx'
p.show()

Generate the simulation grid and inspect it:

In [ ]:
xmin, xmax, ymin, ymax, zmin, zmax = surf.bounds
# set grid and geometry
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax, Nx, Ny, Nz, 
                stl_solids=stl_solids, 
                stl_materials=stl_materials,
                stl_scale=stl_scale,
                stl_rotate=stl_rotate,
                stl_translate=stl_translate)

In [ ]:
grid.inspect()

Add the beam source and define other solver parameters:

In [ ]:
# ------------ Beam source ----------------
# Beam parameters
sigmaz = 10e-2      #[m] -> 2 GHz
q = 1e-9            #[C]
beta = 1.0          # beam beta 
xs = 0.             # x source position [m]
ys = 0.             # y source position [m]
xt = 0.             # x test position [m]
yt = 0.             # y test position [m]
# [DEFAULT] tinj = 8.53*sigmaz/c_light  # injection time offset [s] 

# Simualtion
wakelength = 10. #[m]
add_space = 10   # no. cells

wake = WakeSolver(q=q, sigmaz=sigmaz, beta=beta,
            xsource=xs, ysource=ys, xtest=xt, ytest=yt,
            add_space=add_space, save=False,
            logfile=False)

# ----------- Solver & Simulation ----------
# boundary conditions``
bc_low=['pec', 'pec', 'pec']
bc_high=['pec', 'pec', 'pec']

solver = SolverFIT3D(grid, wake, #dt=dt,
                     bc_low=bc_low, bc_high=bc_high, 
                     use_stl=True, bg='pec')
# Plot settings
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('name', plt.cm.jet(np.linspace(0.0, 0.9))) # CST's colormap

# For plotting
from wakis.sources import Beam
beam = Beam(q=q, sigmaz=sigmaz, beta=beta,
            xsource=xs, ysource=ys)

Inspect the resulting material tensors, for example the permittivity $\varepsilon ^{-1}$

In [ ]:
solver.ieps.inspect(plane='XZ')

Prepare 2D plot settings for on-the-fly plots

In [ ]:
# Plot settings
n=0
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('name', plt.cm.jet(np.linspace(0.05, 0.9))) # CST's colormap

plotkw2D = {'title':'img/E_Abs_2d', 
            'add_patch':['cavity'], 'patch_alpha':0.9,
            'patch_reverse' : True, 
            'vmin':0, 'vmax':700,
            'interpolation' : 'gaussian',
            'cmap' : 'turbo',
            'plane': [slice(0, Nx), int(Ny/2), slice(0, Nz)]}

#solver.plot2D('E', component='Abs', **plotkw2D, off_screen=True, n=n)

Run a simulation generating plots every 20 timesteps after the beam has been injected (around timestep 600)

In [ ]:
Nt = 880            
for n in tqdm(range(Nt)):

    beam.update(solver, n*solver.dt)
    solver.one_step()

    if n%20 == 0 and n>600:
       solver.plot2D('E', component='Abs', **plotkw2D, off_screen=True, n=n)

Convert the frames you generated into a gif with image-magick

In [ ]:
!convert -delay 10 -loop 0 img/E_Abs_2d*.png CEI_logo.gif

Once the simulation is finished, plot the result in a 3D plot:
* Interpolating to the STL cavity
* Or simply catting the simulation domain

In [ ]:
solver.plot3DonSTL('E', component='Abs', cmap=cmap, clim=[0, 700],
    stl_with_field='cavity', field_opacity=1.0,
    stl_transparent=['letters'], stl_opacity=0.8, stl_colors='blue',
    clip_plane=True, clip_normal='-y', clip_origin=[0,0,0],
    off_screen=False, zoom=1.2, n=n, title='img/E_Abs_3d')

In [ ]:
bounds = [xmin, xmax, 0, ymax, zmin, zmax]
solver.plot3D('E', component='Abs', cmap=cmap, clim=[0, 1000],
        add_stl='letters', stl_opacity=1.0, stl_colors='white',
        clip_box=True, clip_bounds=bounds, 
        off_screen=True, zoom=1.2, n=n, title='005_lossy_fancy_cavity/misc/Ez3d0')

You can also inspect the other components of the fields, e.g. the Electric field:

In [ ]:
solver.E.inspect(cmap='bwr')

In [ ]:
for n in tqdm(range(Nt,Nt+5)):

    beam.update(solver, n*solver.dt)
    solver.one_step()

Nt +=5

More plots in 2D:

In [ ]:
# Plot settings
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('name', plt.cm.jet(np.linspace(0.05, 0.9))) # CST's colormap

plotkw2D = {'title':'005_lossy_fancy_cavity/misc/Ez2dv2', 
            'add_patch':['cavity'], 'patch_alpha':0.9,
            'patch_reverse' : True, 
            'vmin':0, 'vmax':700,
            'interpolation' : 'gaussian',
            'cmap' : cmap,
            'plane': [slice(0, Nx), int(Ny/2), slice(0, Nz)]}

solver.plot2D('E', component='Abs', **plotkw2D, off_screen=True, n=n)

Run a few more timesteps if you want: